# Crossmatching NASA Exoplanet Archive data with Gaia DR3 using Gaia DR2 IDs

The Nasa Exoplanet Archive (NEA; https://exoplanetarchive.ipac.caltech.edu/) contains the most up to date information on exoplanet properties. We wish to crossmatch data in the NEA with the most recent data in the Gaia archive (https://gea.esac.esa.int/archive/), which as of 4/11/22 is DR3. The NEA contains Gaia IDs for the exoplanet host stars. However, as of 4/11/22, the NEA Gaia IDs are from Gaia DR2. Unfortunately, the Gaia IDs for the same stars are not neccessarily the same between different data releases. Therefore, we need to carefully crossmatch the Gaia DR2 IDs in the NEA with the correct Gaia DR3 stars.

The https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_auxiliary_tables/ssec_dm_dr2_neighbourhood.html

# Setup 

## Importing modules

In [1]:
import pandas as pd
import re
from astroquery.gaia import Gaia

## Downloading and reading in NEA data

We use the most up to date data from the NEA compositve planetary data: https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PSCompPars. We download all the columns from the most recent data from the NEA compositve planetary catalogue as a csv, which produces a file with a name like, PSCompPars_2022.10.29_03.02.41.csv, where the numbers correspond to the date and time the data were downloaded. Something in the header of the csv file meant pandas could not read the data properly, so the quick work around is to remove all the extraneous header info (all rows starting with #).

In [3]:
# Read in the data using pandas
# exoplanets = pd.read_csv("PSCompPars_2022.10.29_03.02.41.csv", skiprows=315, low_memory=False)  # skipping the first 315 rows which are header information
# exoplanets = pd.read_csv("PSCompPars_2022.10.29_03.02.41.csv", skiprows=315)  # skipping the first 315 rows which are header information
# exoplanets = pd.read_csv("PSCompPars_2022.10.29_03.02.41.csv", skiprows=315)  # skipping the first 315 rows which are header information
# exoplanets = pd.read_csv("PSCompPars_2022.11.04_06.38.57.csv", skiprows=94)
# exoplanets = pd.read_csv("PSCompPars_2022.11.04_06.38.57_nohead.csv")
exoplanets = pd.read_csv("PSCompPars_2022.11.04_06.46.09.csv")

In [4]:
# exoplanets

In [5]:
gaia_id_number = pd.DataFrame(exoplanets['gaia_id'].str.split(expand=True)[2])

In [6]:
gaia_id_number.columns=['gaia_id_number']

In [7]:
gaia_id_number

,gaia_id_number
0,3946945413106333696
1,1696798367260229376
2,1920113512486282240
3,1385293808145621504
4,2135550755683407232
...,...
5192,348020448377061376
5193,348020448377061376
5194,348020448377061376
5195,3794167001116433152


In [8]:
#gaia_id_number.to_csv("PSCompPars_2022.11.04_06.46.09_gaia_id_drop_number_only.csv", index=False)

In [9]:
# Dropping rows with no Gaia ids as this will just complicate the analysis.
gaia_id_number.dropna().to_csv("PSCompPars_2022.11.04_06.46.09_gaia_id_number.csv", index=False)

In [10]:
# exoplanets['gaia_id'].to_csv("gaia_full_name_test.csv", index=False)

# Login to Gaia Archive

In [11]:
#### We can log onto the Gaia archive using the following formats:
#### Gaia.login_gui() - will display window to provide a username and password
#### Gaia.login() - a prompt will ask the user for name and password
#### Gaia.login(user=[username], password=[password]) - just type the username and password in the given fields
#### Gaia.login(credentials_file=[filename] - A path to the file that contains username and password. The username and password must be in different lines!

In [12]:
Gaia.login()

INFO: Login to gaia TAP server [astroquery.gaia.core]


User:  slongmor
Password:  ·········


OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK


# Upload the Gaia DR2 ids to the Gaia archive

Now we need to upload the csv file we created above with Gaia DR2 ID's from the NASA Exoplanets Archive. This can be done on the Gaia archive website using the GUI. Cannot upload the same table twice unless we change the name as they are unique.

In [14]:
# Gaia.upload_table(upload_resource="PSCompPars_2022.11.04_06.46.09_gaia_id_number_dropna.csv", table_name="PSCompPars_2022.11.04_06.46.09_gaia_id_number_dropna.csv", format="CSV")
# Gaia.upload_table(upload_resource="PSCompPars_2022.11.04_06.46.09_gaia_id_number.csv", table_name="PSCompPars_2022.11.04_06.46.09_gaia_id_number.csv", format="CSV")
Gaia.upload_table(upload_resource="PSCompPars_2022.11.04_06.46.09_gaia_id_number.csv", table_name="gaia_dr2_NEA_names", format="CSV")

Sending file: PSCompPars_2022.11.04_06.46.09_gaia_id_number.csv
Uploaded table 'gaia_dr2_NEA_names'.


# Submit crossmatch query

We need to provide a gaia table name to query as well as the username that will be used to load the uploaded table.
Set a name for the file to which queried data will be saved.

In [15]:
table = "gaiadr3.dr2_neighbourhood"
username = "slongmor"
filename = "gaiadr3.dr2_neighbourhood.csv"
# mytable = "PSCompPars_2022.11.04_06.46.09_gaia_id_number.csv"
mytable = "gaia_dr2_NEA_names"

In [16]:
query = f"""
SELECT dr2_names.*, both_names.*
FROM user_{username}.{mytable} AS dr2_names
JOIN {table} as both_names 
    ON both_names.dr2_source_id = dr2_names.gaia_source_id
WHERE ABS(magnitude_difference) < 0.1
"""

In [17]:
Gaia.launch_job_async(query).get_results().to_pandas().to_csv(f"{filename}", index=False)

500 Error 500:
null


HTTPError: Error 500:
null